In [2]:
import pandas as pd
import numpy as np
from numpy.lib import recfunctions as rfn
from sklearn.cluster import MiniBatchKMeans
import cv2
from scipy.cluster.vq import kmeans,vq
from scipy.spatial import distance
import random
from sklearn.preprocessing import MinMaxScaler,minmax_scale
import pickle
import imutils

In [3]:
!pip install opencv-contrib-python==4.4.0.44

     |████████████████████████████████| 55.7MB 79kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


## Dataset Preparation

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import glob
mypath = "/content/drive/My Drive/DIP/DIP_flowers"
# l = glob.glob("/content/drive/My Drive/DIP/DIP_flowers/image_*")
# random.shuffle(l)
# random.shuffle(l)
# random.shuffle(l)
# ll = [pd.read_csv(path) for path in l]
from os import walk
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break
for i in range(len(f)):
  f[i] = mypath + "/" + f[i]
df = pd.DataFrame(f,columns=['img'])
# f = open("/content/drive/My Drive/DIP/DIP_flowers/txt/files.txt")
# data = f.read().splitlines()

In [7]:

df['label']=''
df['label'].iloc[0:80]="Daffodil"
df['label'].iloc[80:160]="Snowdrop"
df['label'].iloc[160:240]="LillyValley"
df['label'].iloc[240:320]="Bluebell"
df['label'].iloc[320:400]="Crocus"
df['label'].iloc[400:480]="Iris"
df['label'].iloc[480:560]="Tigerlily"
df['label'].iloc[560:640]="Tulip"
df['label'].iloc[640:720]="Fritillary"
df['label'].iloc[720:800]="Sunflower"
df['label'].iloc[800:880]="Daisy"
df['label'].iloc[880:1040]="Dandelion"
df['label'].iloc[1040:1120]="Cowslip"
df['label'].iloc[1120:1200]="Buttercup"
df['label'].iloc[1200:1280]="Windflower"
df['label'].iloc[1280:1360]="Pansy"

In [8]:
df

,img,label
0,/content/drive/My Drive/DIP/DIP_flowers/image_...,Daffodil
1,/content/drive/My Drive/DIP/DIP_flowers/image_...,Daffodil
2,/content/drive/My Drive/DIP/DIP_flowers/image_...,Daffodil
3,/content/drive/My Drive/DIP/DIP_flowers/image_...,Daffodil
4,/content/drive/My Drive/DIP/DIP_flowers/image_...,Daffodil
...,...,...
1355,/content/drive/My Drive/DIP/DIP_flowers/image_...,Pansy
1356,/content/drive/My Drive/DIP/DIP_flowers/image_...,Pansy
1357,/content/drive/My Drive/DIP/DIP_flowers/image_...,Pansy
1358,/content/drive/My Drive/DIP/DIP_flowers/image_...,Pansy


In [9]:

df['label'].value_counts()

Dandelion      160
Tulip           80
Tigerlily       80
Buttercup       80
Snowdrop        80
Crocus          80
Sunflower       80
Pansy           80
Windflower      80
Daffodil        80
LillyValley     80
Bluebell        80
Cowslip         80
Iris            80
Daisy           80
Fritillary      80
Name: label, dtype: int64

In [10]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['label'], test_size=0.2, random_state=0, stratify=df['label'])

In [11]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((1088, 2), (272, 2), (1088,), (272,))

In [12]:

X_train['label'].value_counts()

Dandelion      128
Tulip           64
Tigerlily       64
Buttercup       64
Windflower      64
Daffodil        64
LillyValley     64
Bluebell        64
Snowdrop        64
Cowslip         64
Iris            64
Crocus          64
Sunflower       64
Daisy           64
Pansy           64
Fritillary      64
Name: label, dtype: int64

In [13]:
X_test['label'].value_counts()

Dandelion      32
Iris           16
Bluebell       16
Buttercup      16
Windflower     16
Snowdrop       16
Cowslip        16
Pansy          16
Daffodil       16
Crocus         16
LillyValley    16
Tulip          16
Fritillary     16
Sunflower      16
Tigerlily      16
Daisy          16
Name: label, dtype: int64

In [14]:


file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/img.pkl', 'rb')
# dump information to that file
data_gray=pickle.load(file)
# close the file
file.close()
X_train_img_arr_gray,X_test_img_arr_gray = train_test_split(data_gray, test_size=0.2, random_state=0, stratify=df['label'])

In [15]:
file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/all_img_color.pkl', 'rb')
# dump information to that file
data=pickle.load(file)
# close the file
file.close()

X_train_img_arr,X_test_img_arr = train_test_split(data, test_size=0.2, random_state=0, stratify=df['label'])

In [16]:
print(len(X_train_img_arr_gray))
print(len(X_test_img_arr_gray))
print(len(X_train_img_arr))
print(len(X_test_img_arr))

1088
272
1088
272


In [17]:
# X_train_img_arr=[]
# for i in X_train['img']: 
#   img = cv2.imread(i)
#   # print(img)
#   # print(img.shape)
#   X_train_img_arr.append(img)

# X_train_img_arr_gray=[]
# for i in X_train_img_arr: 
#   g=cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   X_train_img_arr_gray.append(g)

# X_test_img_arr=[]
# for i in X_test['img']: 
#   img = cv2.imread(i)
#   X_train_img_arr.append(img)

# X_test_img_arr_gray=[]
# for i in X_test_img_arr:
#   g=cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   X_test_img_arr_gray.append(g)




In [ ]:
#Replace this code with proper image loading code from gdrive as following
#Load images as tuples (img, label)
#split into train test pair after shuffling
#trainX, trainY, testX, testY is obtained

#_________________________________________________#

# imgs = []
# for i in range(1,11):
#   name = 'image_{:04d}.jpg'.format(i)
#   im = cv2.imread(name,0)
#   #im = im.astype('uint8')
#   imgs.append((im, 0))

# for i in range(89,103):
#   im = cv2.imread('image_{:04d}.jpg'.format(i),0)
#   #im = im.astype('uint8')
#   imgs.append((im, 1))

# random.shuffle(imgs)
# train = list(zip(*imgs[:18]))
# trainX = train[0]
# trainY = train[1]

# test = list(zip(*imgs[18:]))
# testX = test[0]
# testY = test[1]

##Shape based Feature Extraction

In [18]:
def FeatureExtractor (imgs, extractor):
  imgsDescriptors = []
  allDescriptors = []
  for img in imgs:
    keypoint, descriptor = extractor.detectAndCompute(img,None)
    # print(descriptor)
    imgsDescriptors.append(descriptor)
    allDescriptors.extend(descriptor)

  return (allDescriptors, imgsDescriptors)



def normalize(hist):
  csum = np.sum(hist, axis = 1).reshape(-1,1)
  # print(csum)
  hist = hist / csum
  # print (hist.shape, hist[0].shape)
  # print(np.sum(hist, axis = 1))
  return list(hist)

# def load_img(xtrain):
#   if(len(images)!=0)
#     images=color_read(xtrain)
#   img_arr=gray_read(images)
#   # img_arr_train=[]
#   # for i in xtrain['img'][:20]: 
#   #     path = i
#   #     img = cv2.imread(path,0)
#   #     # print(img)
#   #     # print(img.shape)
#   #     img_arr_train.append(img)
#   return img_arr

def CreateVocabulary (k, descriptors):  
  #convert descriptors to float?
  vocabulary, distortion = kmeans(descriptors, k, 1)
  return vocabulary

def ComputeHistogram (imgsDescriptors, vocabulary, k):
  histograms = []
  for imgDescriptors in imgsDescriptors:
    histogram = np.zeros(k)
    words, dist = vq(imgDescriptors, vocabulary)
    for w in words:
      histogram[w] += 1
    # histogram = histogram.reshape(-1,1)
    histograms.append(histogram)
 
  return histograms

def Predict (testHistograms, trainHistograms, trainY):
  testY = []
  for test in testHistograms:
    #use eucledian dist for now, change to chi square later
    dists = np.square(test[np.newaxis,:] - trainHistograms).sum(axis=1)
    mostLike = np.argmin(dists)
    testY.append(trainY.iloc[mostLike])
  return testY

In [36]:
sift = cv2.SIFT_create()

# print(images)
(allDescriptors, imgsDescriptorsTrain) = FeatureExtractor(X_train_img_arr_gray, sift)


# -----------------------Pickle code--------------------------
# open a file, where you ant to store the data
file = open('Shape_allDescriptors.pkl', 'wb')
# dump information to that file
pickle.dump(allDescriptors, file)
# close the file
file.close()

file = open('Shape_imgsDescriptionTrain.pkl', 'wb')
# dump information to that file
pickle.dump(imgsDescriptorsTrain, file)
# close the file
file.close()
# ---------------------------------------------------------------

k = 300
vocabulary = CreateVocabulary(k, allDescriptors)
histTrain = ComputeHistogram(imgsDescriptorsTrain, vocabulary, k)
histTrain=normalize(np.array(histTrain))


(ignore, imgsDescriptorsTest) = FeatureExtractor(X_test_img_arr_gray, sift)
histTest = ComputeHistogram(imgsDescriptorsTest, vocabulary, k)
histTest=normalize(np.array(histTest))


# -----------------------Pickle code--------------------------
# open a file, where you ant to store the data
file = open('Shape_hist_train.pkl', 'wb')
# dump information to that file
pickle.dump(histTrain, file)
# close the file
file.close()

file = open('Shape_hist_test.pkl', 'wb')
# dump information to that file
pickle.dump(histTest, file)
# close the file
file.close()
# ---------------------------------------------------------------

In [37]:
pred_y=Predict(histTest, histTrain, y_train)

In [38]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred_y))

0.45588235294117646


In [22]:
#Testing on single image
# im = cv2.imread('image_0001.jpg',0)
# sift = cv2.SIFT_create()
# (allDescriptors, imgsDescriptors) = FeatureExtractor([im], sift)
# k = 20
# vocabulary = CreateVocabulary(k, allDescriptors)
# histograms = ComputeHistogram(imgsDescriptors, vocabulary, k)

In [ ]:
# def FeatureExtractor (classToImgs, extractor):
#   #inputs images in the form {classLabel: [images]}
#   #outputs ( [all descriptors from all the images], {classLabel:[keypoints x 128 ndarray for each classLabel image]} )
#   classToImgDescriptors = {}
#   allDescriptors = []

#   for classLabel,images in classToImgs.items():
#     classToImgDescriptors[classLabel] = []
#     for img in images:
#       keypoint, descriptor = extractor.detectAndCompute(img,None)
#       classToImgDescriptors[classLabel].append(descriptor)
#       allDescriptors.extend(descriptor)

#   return (allDescriptors, classToImgDescriptors)



# def ComputeHistogram (classToImgDescriptors,vocabulary, k):
#   classToImageHistograms = {}
#   for classes, imgs in classToImgDescriptors.items():
#     classToImageHistograms[classes] = []
#     for img in imgs:
#       histogram = np.zeros(k)
#       words, dist = vq(img, vocabulary)
#       for w in words:
#         histogram[w] += 1
#       classToImageHistograms[classes].append(histogram)

##Texture based Feature Extraction

### Using LBP(Local Binary Patterns)

In [39]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern

In [40]:
# def load_image():
#     strr = "Datasets/*.jpg"
#     for file in glob.glob(strr):
#         img=np.asarray(plt.imread(file))
#         arr.append(img)
#     return arr

def preprocessing(df):
    arr_prep=[]
    for i in range(len(df)):
        img=cv2.cvtColor(df['img'][i],0)
        arr_prep.append(img)
    return arr_prep

def extractLBP(img):
    lbp = local_binary_pattern(img, 24,3, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, 27),range=(0, 26))                   
    hist = hist.astype("float")                         
    hist /= (hist.sum() + (1e-7))
    return lbp,hist    

def featureExtraction(arr):
    arr_feature=[]
    vector_feature=[]
    for i in range(len(arr)):
        lb,vektor = extractLBP(arr[i])
        # print(lb.shape)
        arr_feature.append(lb)
        vector_feature.append(vektor)
    return arr_feature, vector_feature


def normalize(hist):
  csum = np.sum(hist, axis = 1).reshape(-1,1)
  # print(csum)
  hist = hist / csum
  # print (hist.shape, hist[0].shape)
  # print(np.sum(hist, axis = 1))
  return list(hist)

def Predict (testHistograms, trainHistograms, trainY):
  testY = []
  for test in testHistograms:
    #use eucledian dist for now, change to chi squaallDescriptors, imgsDescriptorsre later
    dists = np.square(test[np.newaxis,:] - trainHistograms).sum(axis=1)       #using normal distance
    # dist = cv2.compareHist(test,train, cv2.HISTCMP_CHISQR)
    mostLike = np.argmin(dists)
    # print(mostLike)
    # print("---",trainY.iloc[mostLike])
    testY.append(trainY.iloc[mostLike])
  return testY


In [41]:
dataExtracted,vector_hist_train = featureExtraction(X_train_img_arr_gray)
vector_hist_train=normalize(vector_hist_train)

In [42]:
# arr_test= []
# arr_test=load_img(X_test)    #shape feature's load function used 
dataExtracted,vector_hist_test = featureExtraction(X_test_img_arr_gray)
vector_hist_test=normalize(vector_hist_test)


# # -----------------------Pickle code--------------------------
# # open a file, where you ant to store the data
# file = open('Texture_hist_train.pkl', 'wb')
# # dump information to that file
# pickle.dump(vector_hist_train, file)
# # close the file
# file.close()

# file = open('Texture_hist_test.pkl', 'wb')
# # dump information to that file
# pickle.dump(vector_hist_test, file)
# # close the file
# file.close()
# # ---------------------------------------------------------------


In [46]:
# print('Extraction Result')
# fig = plt.figure(figsize=(10,10))
# ax1 = fig.add_subplot(4,2,1)
# ax1.set_title('Before')
# ax1.set_axis_off()
# ax1.imshow(arr_prep[0])

# ax2 = fig.add_subplot(4,2,2)
# ax2.set_title('After')
# ax2.set_axis_off()
# ax2.imshow(dataExtracted[2],cmap=plt.cm.gray)
# plt.show()

In [ ]:
# print("Vector of Image 1 :",vector[0])
# print()
# print("Vector of Image 2 :",vector[1])
# print()
# print("Vector of Image 3 :",vector[2])

In [43]:
pred_y=Predict(vector_hist_test,vector_hist_train,y_train)
# print(pred_y)
# print(y_test[0:20])

In [44]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred_y))

0.14338235294117646


## Color Based Extraction:


In [35]:
def FeatureExtractor (imgs):
  imgsDescriptors = []
  allDescriptors = []
  for img in imgs:
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype('double')
    # print(list(hsv_img[:,:,0]))
    # print(hsv_img[:,:,0].flatten().tolist())
    # print(hsv_img[:,:].flatten().tolist())
    x =rfn.unstructured_to_structured(hsv_img)
    # print(x.flatten().tolist()[0:5])
    x=x.flatten().tolist()
    imgsDescriptors.append(x)
    allDescriptors.extend(x)
  return (allDescriptors, imgsDescriptors)

def Resize(imgs,dim=50):
  new_imgs=[]
  for im in imgs:
    if im.shape[0] < im.shape[1]:
      resized = imutils.resize(im, height=dim) 
    else:
      resized = imutils.resize(im, width=dim)
    # print(resized.shape)
    new_imgs.append(resized)
  return new_imgs



def CreateVocabulary (k, descriptors):  
  #convert descriptors to float?
  # print(descriptors)
  vocabulary=MiniBatchKMeans(n_clusters=k, batch_size = 1000000).fit(descriptors)
  # vocabulary, distortion = kmeans(descriptors, k, 1, check_finite=False)
  # vocabulary, distortion = kmeans(descriptors, k, 1)
  return vocabulary.cluster_centers_
  # return vocabulary

def ComputeHistogram (imgsDescriptors, vocabulary, k):
  histograms = []
  for imgDescriptors in imgsDescriptors:
    histogram = np.zeros(k)
    words, dist = vq(imgDescriptors, vocabulary)
    for w in words:
      histogram[w] += 1
    # histogram = histogram.reshape(-1)
    histograms.append(histogram)
  return histograms

def Predict (testHistograms, trainHistograms, trainY):
  testY = []
  for test in testHistograms:
    #use eucledian dist for now, change to chi squaallDescriptors, imgsDescriptorsre later
    dists = np.square(test[np.newaxis,:] - trainHistograms).sum(axis=1)       #using normal distance
    # dist = cv2.compareHist(test,train, cv2.HISTCMP_CHISQR)
    mostLike = np.argmin(dists)
    # print(mostLike)
    # print("---",trainY.iloc[mostLike])
    testY.append(trainY.iloc[mostLike])
  return testY



In [36]:
from skimage.color import rgb2hsv

# X_train_img_arr=[]
# for i in X_train['img'][:1]: 
#   img = cv2.imread(i)
#   # print(img)
#   # print(img.shape)
#   X_train_img_arr.append(img)

X_train_img_arr=Resize(X_train_img_arr)

allDescriptors_train, imgsDescriptors_train=FeatureExtractor(X_train_img_arr)


# # -----------------------Pickle code--------------------------
# # open a file, where you ant to store the data
# file = open('Color_All_Descriptors.pkl', 'wb')
# # dump information to that file
# pickle.dump(allDescriptors_train, file)
# # close the file
# file.close()

# file = open('Color_Image_Descriptors.pkl', 'wb')
# # dump information to that file
# pickle.dump(imgsDescriptors_train, file)
# # close the file
# file.close()
# # ---------------------------------------------------------------






In [37]:

k = 300
print("Features Extracted train")
vocabulary = CreateVocabulary(k, allDescriptors_train)
print("Vocabulary Created train")
hist_train = ComputeHistogram(imgsDescriptors_train, vocabulary, k)
hist_train=normalize(np.array(hist_train))
# print("Histogram ban gaya train")
# # print(hist_train)




Features Extracted train
Vocabulary Created train


In [23]:
# -----------------------Pickle code--------------------------
# open a file, where you ant to store the data
file = open('Color_AllDescriptorsTrain', 'wb')
# dump information to that file
pickle.dump(allDescriptors_train, file)
# close the file
file.close()

file = open('Color_ImgDescriptorsTrain', 'wb')
# dump information to that file
pickle.dump(imgsDescriptors_train, file)
# close the file
file.close()
# file = open('important', 'rb') # dump information to that file 
# data = pickle.load(file)       # close the file 
# file.close()
# ---------------------------------------------------------------


In [38]:

X_test_img_arr=Resize(X_test_img_arr)

allDescriptors_test, imgsDescriptors_test=FeatureExtractor(X_test_img_arr)
hist_test = ComputeHistogram(imgsDescriptors_test, vocabulary, k)
hist_test=normalize(np.array(hist_test))
# for i in hist_train:
#   print(i)
# print(type(hist_train))



color_hist_train=hist_train
color_hist_test=hist_test



In [25]:
# -----------------------Pickle code--------------------------

file = open('Color_AllDescriptorsTest.pkl', 'wb')
# dump information to that file
pickle.dump(allDescriptors_test, file)
# close the file
file.close()

file = open('Color_ImgDescriptorsTest.pkl', 'wb')
# dump information to that file
pickle.dump(imgsDescriptors_test, file)
# close the file
file.close()


# open a file, where you ant to store the data
file = open('Color_Histogram_train.pkl', 'wb')
# dump information to that file
pickle.dump(hist_train, file)
# close the file
file.close()

file = open('Color_Histogram_test.pkl', 'wb')
# dump information to that file
pickle.dump(hist_test, file)
# close the file
file.close()
# file = open('Color_Histogram_test.pkl', 'wb')
# # dump information to that file
# pickle.dump(hist_test, file)
# # close the file
# file.close()


# ---------------------------------------------------------------


In [28]:
pred_y=Predict(hist_test,hist_train,y_train)
# print(pred_y)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred_y))


0.3161764705882353


In [40]:
def Predict (testHistogramsS, trainHistogramsS, testHistogramsT, trainHistogramsT,testHistogramsC, trainHistogramsC, trainY, a1=1, a2=1, a3=0.4):
  testY = []
  for testS, testT, testC in zip(testHistogramsS,testHistogramsT, testHistogramsC):
    #use eucledian dist for now, change to chi squaallDescriptors, imgsDescriptorsre later
    distsS = np.square(testS[np.newaxis,:] - trainHistogramsS).sum(axis=1)  
    distsT = np.square(testT[np.newaxis,:] - trainHistogramsT).sum(axis=1)  
    distsC = np.square(testC[np.newaxis,:] - trainHistogramsC).sum(axis=1) 
    print()
    dists = a1 *  distsS + a2* distsT + a3 * distsC
    mostLike = np.argmin(dists)
    # print(mostLike)
    # print("---",trainY.iloc[mostLike])
    testY.append(trainY.iloc[mostLike])
  return testY



In [31]:
file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/Shape_hist_train.pkl', 'rb') # dump information to that file 
shape_hist_train= pickle.load(file)       # close the file 
file.close()

file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/Shape_hist_test.pkl', 'rb') # dump information to that file 
shape_hist_test= pickle.load(file)       # close the file 
file.close()


file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/Texture_hist_test.pkl', 'rb') # dump information to that file 
texture_hist_test= pickle.load(file)       # close the file 
file.close()


file = open('/content/drive/MyDrive/DIP/DIP_flowers/data/Texture_hist_train.pkl', 'rb') # dump information to that file 
texture_hist_train= pickle.load(file)       # close the file 
file.close()



In [55]:
pred_combi=Predict(shape_hist_test[-272:],shape_hist_train,texture_hist_test[-272:],texture_hist_test,color_hist_test,color_hist_train[0:952],y_train)

272


In [ ]:
pred_y=Predict(hist_test,hist_train,y_train)
# print(pred_y)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred_combi))